In [3]:
import pandas as pd
import numpy as np
from io import StringIO
import html
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Define data folder
data_folder = os.path.join('../..', 'data/row')

# Define file paths for Dataset 2 (Resources & Potentials)
files_dataset2 = {
    # Size of economy (we'll keep first 3 columns for country info)
    "size_economy": os.path.join(data_folder, "WV.1_Size_of_the_economy.xls"),
    
    # Environment files
    "rural_environment": os.path.join(data_folder, "3.1_Rural_environment_and_land_use.xls"),
    "agricultural_inputs": os.path.join(data_folder, "3.2_Agricultural_inputs.xls"),
    "deforestation": os.path.join(data_folder, "3.4_Deforestation_and_biodiversity.xls"),
    "energy": os.path.join(data_folder, "3.6_Energy_production_and_use.xls"),
    "natural_resources": os.path.join(data_folder, "3.14_Contribution_of_natural_resources_to_gross_domestic_product.xls"),
    
    # People files (human capital)
    "population": os.path.join(data_folder, "2.1_Population_dynamics.xls"),
    "labor_force": os.path.join(data_folder, "2.2_Labor_force_structure.xls"),
    "education_participation": os.path.join(data_folder, "2.8_Participation_in_education.xls"),
    "education_completion": os.path.join(data_folder, "2.10_Education_completion_and_outcomes.xls")
}

print("✅ File paths defined")
print(f"📁 Total files to load: {len(files_dataset2)}")

✅ File paths defined
📁 Total files to load: 10


In [5]:
def load_and_clean_table(path, skip_cols=0):
    """
    Load and clean World Bank data tables
    
    Parameters:
    - path: file path
    - skip_cols: number of data columns to skip (after Country column)
    """
    try:
        # First try reading as HTML disguised as XLS
        with open(path, 'r', encoding='utf-8', errors='ignore') as f:
            html_content = f.read()

        dfs = pd.read_html(StringIO(html_content))
        df = dfs[0]

    except Exception:
        # Fallback: real Excel (just in case)
        df = pd.read_excel(path, header=[0,1,2,3])

    # Handle multi-index or normal headers
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [
            '_'.join([str(c) for c in col if 'Unnamed' not in str(c)])
            for col in df.columns
        ]
    else:
        df.columns = df.columns.astype(str)

    # Rename first column → Country
    df.rename(columns={df.columns[0]: 'Country'}, inplace=True)

    # Skip first N data columns (except Country)
    if skip_cols > 0:
        keep = ['Country'] + list(df.columns[1 + skip_cols:])
        df = df[keep]

    # Replace World Bank missing marker
    df.replace('..', pd.NA, inplace=True)

    # Convert numeric columns
    for col in df.columns:
        if col != 'Country':
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

print("✅ Load and clean function defined")

✅ Load and clean function defined


In [6]:
print("\n" + "="*60)
print("📥 LOADING DATASET 2: RESOURCES & POTENTIALS")
print("="*60)

print("\n📊 1. Loading 'Size of Economy' (keeping first 3 columns for country info)...")

# Load size of economy WITHOUT skipping columns (we'll select what we need)
size_economy_full = load_and_clean_table(files_dataset2["size_economy"], skip_cols=0)

# Keep only first 3 data columns after Country
# These typically contain: Population, Surface area, or basic country identifiers
country_info_cols = ['Country'] + list(size_economy_full.columns[1:4])
country_info_df = size_economy_full[country_info_cols].copy()

print(f"   ✅ Loaded: {country_info_df.shape}")
print(f"   Columns kept: {list(country_info_df.columns)}")
print(f"\n   Sample data:")
country_info_df.head()


📥 LOADING DATASET 2: RESOURCES & POTENTIALS

📊 1. Loading 'Size of Economy' (keeping first 3 columns for country info)...
   ✅ Loaded: (227, 4)
   Columns kept: ['Country', 'Population_millions_2024', 'Surface area_sq. km thousands_2024', 'Population density_people per sq. km_2024']

   Sample data:


,Country,Population_millions_2024,Surface area_sq. km thousands_2024,Population density_people per sq. km_2024
0,Afghanistan,42.647492,652.87,63.558501
1,Albania,2.700000,28.75,100.217956
2,Algeria,46.814308,2381.74,19.382560
3,American Samoa,0.046770,0.20,237.605000
4,Andorra,0.081940,0.47,172.034043


In [7]:
print("\n📊 2. Loading Environment files...")

# 2.1 Rural environment and land use
print("   • Loading 3.1 Rural environment and land use...")
rural_env_df = load_and_clean_table(files_dataset2["rural_environment"], skip_cols=0)
print(f"     ✅ Shape: {rural_env_df.shape}")

# 2.2 Agricultural inputs
print("   • Loading 3.2 Agricultural inputs...")
agri_inputs_df = load_and_clean_table(files_dataset2["agricultural_inputs"], skip_cols=0)
print(f"     ✅ Shape: {agri_inputs_df.shape}")

# 2.3 Deforestation and biodiversity
print("   • Loading 3.4 Deforestation and biodiversity...")
deforestation_df = load_and_clean_table(files_dataset2["deforestation"], skip_cols=0)
print(f"     ✅ Shape: {deforestation_df.shape}")

# 2.4 Energy production and use
print("   • Loading 3.6 Energy production and use...")
energy_df = load_and_clean_table(files_dataset2["energy"], skip_cols=0)
print(f"     ✅ Shape: {energy_df.shape}")

# 2.5 Natural resources contribution to GDP
print("   • Loading 3.14 Contribution of natural resources to GDP...")
natural_resources_df = load_and_clean_table(files_dataset2["natural_resources"], skip_cols=0)
print(f"     ✅ Shape: {natural_resources_df.shape}")

print("\n   ✅ All environment files loaded successfully")


📊 2. Loading Environment files...
   • Loading 3.1 Rural environment and land use...
     ✅ Shape: (229, 8)
   • Loading 3.2 Agricultural inputs...
     ✅ Shape: (226, 8)
   • Loading 3.4 Deforestation and biodiversity...
     ✅ Shape: (226, 9)
   • Loading 3.6 Energy production and use...
     ✅ Shape: (226, 14)
   • Loading 3.14 Contribution of natural resources to GDP...
     ✅ Shape: (226, 7)

   ✅ All environment files loaded successfully


In [8]:
print("\n📊 3. Loading People (Human Capital) files...")

# 3.1 Population dynamics
print("   • Loading 2.1 Population dynamics...")
population_df = load_and_clean_table(files_dataset2["population"], skip_cols=0)
print(f"     ✅ Shape: {population_df.shape}")

# 3.2 Labor force structure
print("   • Loading 2.2 Labor force structure...")
labor_force_df = load_and_clean_table(files_dataset2["labor_force"], skip_cols=0)
print(f"     ✅ Shape: {labor_force_df.shape}")

# 3.3 Education participation
print("   • Loading 2.8 Participation in education...")
edu_participation_df = load_and_clean_table(files_dataset2["education_participation"], skip_cols=0)
print(f"     ✅ Shape: {edu_participation_df.shape}")

# 3.4 Education completion and outcomes
print("   • Loading 2.10 Education completion and outcomes...")
edu_completion_df = load_and_clean_table(files_dataset2["education_completion"], skip_cols=0)
print(f"     ✅ Shape: {edu_completion_df.shape}")

print("\n   ✅ All people/human capital files loaded successfully")


📊 3. Loading People (Human Capital) files...
   • Loading 2.1 Population dynamics...
     ✅ Shape: (226, 11)
   • Loading 2.2 Labor force structure...
     ✅ Shape: (226, 10)
   • Loading 2.8 Participation in education...
     ✅ Shape: (226, 13)
   • Loading 2.10 Education completion and outcomes...
     ✅ Shape: (226, 11)

   ✅ All people/human capital files loaded successfully


In [9]:
print("\n🏷️ 4. Adding prefixes for clarity...")

# Add prefixes to all dataframes (except country_info which we keep as is)
rural_env_df = rural_env_df.rename(columns={
    col: f"Rural_Env_{col}" if col != 'Country' else col 
    for col in rural_env_df.columns
})

agri_inputs_df = agri_inputs_df.rename(columns={
    col: f"Agri_Inputs_{col}" if col != 'Country' else col 
    for col in agri_inputs_df.columns
})

deforestation_df = deforestation_df.rename(columns={
    col: f"Forest_Bio_{col}" if col != 'Country' else col 
    for col in deforestation_df.columns
})

energy_df = energy_df.rename(columns={
    col: f"Energy_{col}" if col != 'Country' else col 
    for col in energy_df.columns
})

natural_resources_df = natural_resources_df.rename(columns={
    col: f"NatResource_GDP_{col}" if col != 'Country' else col 
    for col in natural_resources_df.columns
})

population_df = population_df.rename(columns={
    col: f"Population_{col}" if col != 'Country' else col 
    for col in population_df.columns
})

labor_force_df = labor_force_df.rename(columns={
    col: f"Labor_{col}" if col != 'Country' else col 
    for col in labor_force_df.columns
})

edu_participation_df = edu_participation_df.rename(columns={
    col: f"Edu_Participation_{col}" if col != 'Country' else col 
    for col in edu_participation_df.columns
})

edu_completion_df = edu_completion_df.rename(columns={
    col: f"Edu_Completion_{col}" if col != 'Country' else col 
    for col in edu_completion_df.columns
})

print("   ✅ All prefixes added")


🏷️ 4. Adding prefixes for clarity...
   ✅ All prefixes added


In [10]:
print("\n🔗 5. Merging all data...")

# Ensure all 'Country' columns are strings
def ensure_country_string(df):
    """Ensure Country column is string type"""
    if 'Country' in df.columns:
        df['Country'] = df['Country'].astype(str).str.strip()
    return df

# Apply to all dataframes
country_info_df = ensure_country_string(country_info_df)
rural_env_df = ensure_country_string(rural_env_df)
agri_inputs_df = ensure_country_string(agri_inputs_df)
deforestation_df = ensure_country_string(deforestation_df)
energy_df = ensure_country_string(energy_df)
natural_resources_df = ensure_country_string(natural_resources_df)
population_df = ensure_country_string(population_df)
labor_force_df = ensure_country_string(labor_force_df)
edu_participation_df = ensure_country_string(edu_participation_df)
edu_completion_df = ensure_country_string(edu_completion_df)

print("   ✅ All 'Country' columns converted to strings")

# Start with country info
dataset2 = country_info_df.copy()
print(f"   Starting with Country Info: {dataset2.shape[0]} countries, {dataset2.shape[1]-1} columns")

# List of all dataframes to merge
dataframes_to_merge = [
    ("Rural Environment", rural_env_df),
    ("Agricultural Inputs", agri_inputs_df),
    ("Deforestation & Biodiversity", deforestation_df),
    ("Energy Production & Use", energy_df),
    ("Natural Resources to GDP", natural_resources_df),
    ("Population Dynamics", population_df),
    ("Labor Force Structure", labor_force_df),
    ("Education Participation", edu_participation_df),
    ("Education Completion", edu_completion_df)
]

# Merge one by one
merge_count = 0
for name, df in dataframes_to_merge:
    if not df.empty and 'Country' in df.columns:
        try:
            before_countries = dataset2.shape[0]
            before_cols = dataset2.shape[1] - 1
            
            # Check for common countries
            common_countries = set(dataset2['Country']).intersection(set(df['Country']))
            print(f"\n   Merging {name}...")
            print(f"     Common countries: {len(common_countries)}")
            
            # Merge using outer join to keep all countries
            dataset2 = pd.merge(dataset2, df, on='Country', how='outer')
            merge_count += 1
            
            after_countries = dataset2.shape[0]
            after_cols = dataset2.shape[1] - 1
            new_cols = after_cols - before_cols
            
            print(f"     ✅ Successfully merged!")
            print(f"       Countries: {before_countries} → {after_countries}")
            print(f"       Columns: +{new_cols} (total: {after_cols})")
            
        except Exception as e:
            print(f"     ❌ Error merging {name}: {str(e)}")
    else:
        print(f"\n   ⚠️ Skipping {name}: DataFrame is empty or missing 'Country' column")

print(f"\n   📊 Merge completed: {merge_count}/{len(dataframes_to_merge)} dataframes merged")
print(f"   Final dataset shape: {dataset2.shape}")
print(f"   🌍 Total countries/regions: {dataset2.shape[0]}")
print(f"   📈 Total indicators: {dataset2.shape[1] - 1}")


🔗 5. Merging all data...
   ✅ All 'Country' columns converted to strings
   Starting with Country Info: 227 countries, 3 columns

   Merging Rural Environment...
     Common countries: 224
     ✅ Successfully merged!
       Countries: 227 → 232
       Columns: +7 (total: 10)

   Merging Agricultural Inputs...
     Common countries: 226
     ✅ Successfully merged!
       Countries: 232 → 232
       Columns: +7 (total: 17)

   Merging Deforestation & Biodiversity...
     Common countries: 226
     ✅ Successfully merged!
       Countries: 232 → 232
       Columns: +8 (total: 25)

   Merging Energy Production & Use...
     Common countries: 226
     ✅ Successfully merged!
       Countries: 232 → 232
       Columns: +13 (total: 38)

   Merging Natural Resources to GDP...
     Common countries: 226
     ✅ Successfully merged!
       Countries: 232 → 232
       Columns: +6 (total: 44)

   Merging Population Dynamics...
     Common countries: 226
     ✅ Successfully merged!
       Countries: 

In [11]:
print("\n🧹 6. Cleaning merged dataset...")

# Remove any duplicate country rows
before_dedup = dataset2.shape[0]
dataset2 = dataset2.drop_duplicates(subset='Country', keep='first')
after_dedup = dataset2.shape[0]
if before_dedup != after_dedup:
    print(f"   Removed {before_dedup - after_dedup} duplicate country rows")

# Clean country names (remove extra spaces)
dataset2['Country'] = dataset2['Country'].str.strip()

# Replace any remaining '..' with NaN
dataset2 = dataset2.replace('..', pd.NA)

# Sort by country name
dataset2 = dataset2.sort_values('Country').reset_index(drop=True)

print(f"   ✅ Dataset cleaned and sorted")
print(f"   Final shape: {dataset2.shape}")

# Display preview
print("\n   📋 Dataset Preview:")
dataset2.head()


🧹 6. Cleaning merged dataset...
   ✅ Dataset cleaned and sorted
   Final shape: (232, 86)

   📋 Dataset Preview:


,Country,Population_millions_2024,Surface area_sq. km thousands_2024,Population density_people per sq. km_2024,Rural_Env_Rural population_2022,Rural_Env_Rural population growth (annual %)_2022,Rural_Env_Land area (sq. km)_2021,Rural_Env_Forest area (% of land area)_2021,Rural_Env_Permanent cropland (% of land area)_2021,Rural_Env_Arable land (% of land area)_2021,...,Edu_Completion_Primary completion rate_Male_% of relevant age group_2022,Edu_Completion_Primary completion rate_Female_% of relevant age group_2022,Edu_Completion_Lower secondary completion rate_Male_% of relevant age group_2022,Edu_Completion_Lower secondary completion rate_Female_% of relevant age group_2022,Edu_Completion_Youth literacy rate_Male_% of ages 15-24_2016-23,Edu_Completion_Youth literacy rate_Female_% of ages 15-24_2016-23,Edu_Completion_Adult literacy rate_Male_% ages 15 and older_2016-23,Edu_Completion_Adult literacy rate_Female_% ages 15 and older_2016-23,Edu_Completion_Students at lowest proficiency on PISA_Mathematics_% of 15 year-olds_2018,Edu_Completion_Students at lowest proficiency on PISA_Science_% of 15 year-olds_2018
0,Afghanistan,42.647492,652.87,63.558501,29778.38,1.03,652230.0,1.85,0.34,12.00,...,107.0,70.0,75.0,45.0,83.0,44.0,52.0,27.0,NaN,NaN
1,Albania,2.700000,28.75,100.217956,1005.55,-3.48,27400.0,28.79,3.20,21.89,...,105.0,98.0,99.0,95.0,99.0,99.0,98.0,97.0,17.0,2.0
2,Algeria,46.814308,2381.74,19.382560,11473.04,-0.42,2381741.0,0.82,0.39,3.16,...,93.0,95.0,60.0,90.0,NaN,98.0,NaN,74.0,51.0,4.0
3,American Samoa,0.046770,0.20,237.605000,6.19,-2.02,200.0,85.50,9.65,4.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,0.081940,0.47,172.034043,9.72,2.08,470.0,34.04,0.02,1.59,...,71.0,68.0,100.0,101.0,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print("\n💾 7. Saving Dataset 2...")

# Create meaningful filename: resources_and_human_capital_dataset.csv
output_filename = "../../data/preprocessed/resources_and_human_capital_dataset.csv"

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(output_filename), exist_ok=True)

# Save to CSV
dataset2.to_csv(output_filename, index=False)

print(f"   ✅ Saved as: {output_filename}")

# Verify the file
if os.path.exists(output_filename):
    verify_df = pd.read_csv(output_filename)
    print(f"   📊 Verification: {verify_df.shape[0]} rows, {verify_df.shape[1]} columns")
    print(f"   🌍 Countries count: {verify_df.shape[0]}")
    print(f"   📈 Indicators count: {verify_df.shape[1] - 1}")
    
    # Check for Algeria
    algeria_in_file = verify_df[verify_df['Country'].str.contains('Algeria', case=False, na=False)]
    if not algeria_in_file.empty:
        print(f"   🇩🇿 Algeria in saved file: YES")
    else:
        print(f"   🇩🇿 Algeria in saved file: NO")
else:
    print("   ❌ ERROR: File was not created")

print("\n" + "="*60)
print("✅ DATASET 2 CREATION COMPLETED!")
print("   Dataset Name: Resources and Human Capital Dataset")
print("   Focus: Natural resources, environment, population, education")
print("="*60)


💾 7. Saving Dataset 2...
   ✅ Saved as: ../../data/preprocessed/resources_and_human_capital_dataset.csv
   📊 Verification: 232 rows, 86 columns
   🌍 Countries count: 232
   📈 Indicators count: 85
   🇩🇿 Algeria in saved file: YES

✅ DATASET 2 CREATION COMPLETED!
   Dataset Name: Resources and Human Capital Dataset
   Focus: Natural resources, environment, population, education


In [13]:
for features in dataset2.columns:
    print(features)

Country
Population_millions_2024
Surface area_sq. km thousands_2024
Population density_people per sq. km_2024
Rural_Env_Rural population_2022
Rural_Env_Rural population growth (annual %)_2022
Rural_Env_Land area (sq. km)_2021
Rural_Env_Forest area (% of land area)_2021
Rural_Env_Permanent cropland (% of land area)_2021
Rural_Env_Arable land (% of land area)_2021
Rural_Env_Arable land (hectares per person)_2021
Agri_Inputs_Agricultural land_% of land area_2021
Agri_Inputs_Agricultural land_% irrigated_2021
Agri_Inputs_Average annual precipitation_millimeters_2020
Agri_Inputs_Land under cereal production_hectares thousands_2021
Agri_Inputs_Fertilizer consumption_% of fertilizer production_2021
Agri_Inputs_Fertilizer consumption_kilograms per hectare of arable land_2021
Agri_Inputs_Agricultural employment_% of total employment_2020
Forest_Bio_Forest area_sq. km thousands_1990
Forest_Bio_Forest area_sq. km thousands_2021
Forest_Bio_Threatened species_Mammals_2018
Forest_Bio_Threatened spec